## Generate SVG's for to plot a function on a grid

Input is a function $f: \mathbb{R}^2 \rightarrow \mathbb{R}$, material width $w$, bounding box $[a_x,b_x]\times [a_y,b_y] $, number of plancks in each direction $N_x, N_y$, and depth of join cut $d_c$ (maybe make that adaptive in the future). We want to produce SVG paths for planks which will interlock and effectively graph $f$. 

## IMPORTANT: 
Unfortunately this only works with v0.6 at the moment. The translation of iterators from 0.6 to 0.7+ is not exactly straight forward, at least for something funny like `ClosedCurve` which has cyclic indices

In [1]:

include("glowify_func.jl")

glowify_grid_func (generic function with 2 methods)

There are two coordinate (unit) systems to keep track of - one concrete (for the material), one abstract (for the mathematical). I'll first produce vectors representing the graph along cross sections, and then scale according to the material. 

In [10]:
const LineSamplePoints = StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
struct DiscreteDomain
    Nx::Int
    Ny::Int
    minmin::Point
    maxmax::Point
end

In [11]:
function func_vals_along_grid(f::Function, dd::DiscreteDomain = DiscreteDomain(10,10,Point(0,0),Point(1,1)))::Tuple{LineSamplePoints,LineSamplePoints,Array{Float64,2}}
    mxy = dd.minmin
    Mxy = dd.maxmax
    x_grid_vals = linspace(mxy.x,Mxy.x,dd.Nx)
    y_grid_vals = linspace(mxy.y,Mxy.y,dd.Ny)
    vals_at_intersections = Array{Float64}(dd.Ny,dd.Nx)
    for (i,x) in enumerate(x_grid_vals)
        for (j,y) in enumerate(y_grid_vals)
            vals_at_intersections[j,i] = f(x_grid_vals[i],y_grid_vals[j])
        end
    end
    (y_grid_vals,x_grid_vals,vals_at_intersections)
end

func_vals_along_grid (generic function with 2 methods)

## Some assumptions:
1) The distance between neighboring sample points is a reasonable proxy for the distance length of the cross-section between the two points. That is, the grid points are assumed to be fine enough to basically capture the variations of the function, i.e. the derivative of the function is approximately constant between neighboring grid points

2) The function minimum and maximum are basically achieved at the grid points

In [3]:
f(x,y) = y.^2 - x.^2

f (generic function with 1 method)

In [4]:
f(3,4)

7

## Translating to physical coordinates

The above deals with sampling the function in those coordinates. We need to translate to physical coordinates, which I won't determine, but should be constant.

In [18]:
using PointsNCurves
#const LineSamplePoints = StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
struct DiscreteDomain
    Nx::Int
    Ny::Int
    minmin::Point
    maxmax::Point
end
function glowify_curves_on_grid(vert_curves::Curves, horz_curves::Curves, mat_width::Float64,x_length::Float64, y_length::Float64, z_length::Float64, z_min::Float64)
  # vert_curves will get cut from above
  # horz_curves will get cut from below

  # Assuming even spacing of grid lines.
  # I have in mind that the minimum and maximum x coordinates are uniform over vert_curves and horz_curves
  # Major assumption: the "x" coordinates of the points in vert_curves and horz_curves are monotonically non-decreasing!
  depth_factor = .5
  cut_width = mat_width


  Nvert = length(vert_curves)
  Nhorz = length(horz_curves)

  # we'll scale the values according to z_length and the input min, max of the function
  all_func_max = max(mapreduce(c -> maxy(c),max,vert_curves),mapreduce(c -> maxy(c), max, horz_curves))
  all_func_min = min(mapreduce(c -> miny(c),min,vert_curves),mapreduce(c -> miny(c), min, horz_curves))

  # scale the "y" coordinates of the curves by z_length, and shift by z_min:
  scale_z(c::Curve) = map!(p -> Point(p.x, z_min + z_length*((p.y - all_func_min)/(all_func_max - all_func_min))),c)

  for c in vert_curves
    scale_z(c)
  end
  for c in horz_curves
    scale_z(c)
  end

  # Now the depth axis is scaled as desired.

  # (re)compute the intersection points given the curves.
  horz_input_min = mapreduce(c -> minx(c), min, horz_curves)
  horz_input_max = mapreduce(c -> maxx(c), max, horz_curves)
  vert_input_min = mapreduce(c -> minx(c), min, vert_curves)
  vert_input_max = mapreduce(c -> maxx(c), max, vert_curves)

  horz_disc_vals = linspace(horz_input_min, horz_input_max, Nvert + 2)[2:end-1]
  vert_disc_vals = linspace(vert_input_min, vert_input_max, Nhorz + 2)[2:end-1]

  # Now compute the (minimum) of the depths of the horz_curves and vert_curves where they should intersect:
  horz_vals_along_grid = Array{Float64}(Nvert,Nhorz)
  vert_vals_along_grid = Array{Float64}(Nvert,Nhorz)
  cut_depth_from_bottom = Array{Float64}(Nvert,Nhorz)

  box_curve(c::Curve, xmin, xmax)::ClosedCurve = ClosedCurve(vcat([Point(xmin,0)],c, [Point(xmax,0)]))
  horz_planks = map(c -> box_curve(c,horz_input_min,horz_input_max),horz_curves)
  vert_planks = map(c -> box_curve(c,vert_input_min,vert_input_max),vert_curves)

  horz_cuts = ClosedCurves()
  vert_cuts = ClosedCurves()
  
    # from above
  vert_cutout(x_coord,cut_depth) = ClosedCurve(
    [Point(x_coord-cut_width/2,z_length+z_min+5),
    Point(x_coord+cut_width/2,z_length+z_min+5),
    Point(x_coord+cut_width/2,cut_depth),
    Point(x_coord-cut_width/2,cut_depth)            
    ])
    # from below
  horz_cutout(x_coord,cut_depth) = ClosedCurve(
    [
    Point(x_coord-cut_width/2,-5),
    Point(x_coord+cut_width/2,-5),                      
    Point(x_coord+cut_width/2,cut_depth),            
    Point(x_coord-cut_width/2,cut_depth)                        
    ])

  for (i,horz) in enumerate(horz_curves) # Nhorz of these. They cross the vertical curves
    #cur_horz_idx = 1
    for (j,vert) in enumerate(vert_curves) # Nvert of these. They cross the horizontal curves
    #  cur_vert_idx = 1
      horz_coord = horz_disc_vals[j]
      vert_coord = vert_disc_vals[i]
      # Do horz or verts have values at those coordinates?
      # increment the horizontal and vertial indices until the "x" coordinates 
#       while (horz[cur_horz_idx].x < horz_coord)
#         cur_horz_idx += 1
#       end
#       while (vert[cur_vert_idx].x < vert_coord)
#         cur_vert_idx += 1
#       end
#       min_horz_func_val = horz[cur_horz_idx].y
#       min_vert_func_val = vert[cur_vert_idx].y            
    horzfunccut = vert_cut(horz_coord,z_min)
    ~,~,horz_func_vals_at_intersection = masked_occluded_intersections(horz_planks[i],horzfunccut)
    min_horz_func_val = minimum(map(p -> p.y,horz_func_vals_at_intersection))
    
    vertfunccut = vert_cut(vert_coord,z_min)
    ~,~,vert_func_vals_at_intersection = masked_occluded_intersections(vert_planks[j],vertfunccut)
    min_vert_func_val = minimum(map(p -> p.y,vert_func_vals_at_intersection))
    cut_depth = depth_factor * min(min_horz_func_val, min_vert_func_val)
    if (cut_depth_from_bottom[j,i] < z_min/2)
       error("somehow scaling the function values")
    end
      horz_cut = horz_cutout(horz_coord, cut_depth_from_bottom[j,i])
      push!(horz_cuts,horz_cut)
      (~, occluded_horz_plank, horz_intersections) = masked_occluded_intersections(horz_planks[i],horz_cut)
      if (length(occluded_horz_plank) <= 2)
        horz_planks[i] = ClosedCurve(occluded_horz_plank[1])
      else
        println(occluded_horz_plank[1])
        println("\n\n\n")
        println(occluded_horz_plank[2])
        error("occluded_horz_plank has length $(length(occluded_horz_plank))")
      end
      vert_cut = vert_cutout(vert_coord, cut_depth_from_bottom[j,i])
      push!(vert_cuts,vert_cut)
      (~, occluded_vert_plank, vert_intersections) = masked_occluded_intersections(vert_planks[j],vert_cut)                            
      if (length(occluded_vert_plank) <= 2)
        vert_planks[j] = ClosedCurve(occluded_vert_plank[1])
      else
        println(occluded_vert_plank)
        error("occluded_vert_plank has length $(length(occluded_vert_plank))")
      end
    end # inner for
  end # outer for
  (horz_planks,vert_planks,horz_cuts,vert_cuts)
end

function sample_grid_func(f::Function,Nvert::Int,Nhorz::Int,fsvert::Int, fshorz::Int,vert_min::Float64,vert_max::Float64,horz_min::Float64,horz_max::Float64)
 (Nvert < 3) && error("Need more vertical lines. the sample domain 
    should have 2 more than number of planks in a given direction")
  (Nhorz < 3) && error("Need more horizontal lines. the sample domain 
    should have 2 more than number of planks in a given direction")


  num_horz_sample_points = fshorz
  num_vert_sample_points = fsvert

  horz_x_coords = linspace(horz_min,horz_max,num_horz_sample_points)
  vert_x_coords = linspace(vert_min,vert_max,num_vert_sample_points)

  # "x" values that are constant for the vertical curves
  vert_cross_domain_vals = linspace(horz_min,horz_max,Nvert)[2:end-1]

  # "y" values that are constant for the horizontal curves
  horz_cross_domain_vals = linspace(vert_min,vert_max,Nhorz)[2:end-1]

  vert_curves = Curves(Nvert-2)
  horz_curves = Curves(Nhorz-2)

  for (crv_idx, cross_val) in enumerate(vert_cross_domain_vals)
    crv = []
    for xval in vert_x_coords
      push!(crv,Point(xval, f(cross_val,xval)))
    end
    vert_curves[crv_idx] = crv
  end

  for (crv_idx,cross_val) in enumerate(horz_cross_domain_vals)
    crv = []
    for xval in horz_x_coords
      push!(crv,Point(xval, f(xval,cross_val)))
    end
    horz_curves[crv_idx] = crv
  end

  horz_curves, vert_curves
end

function glowify_grid_func(f::Function,sample_domain::DiscreteDomain,mat_width::Float64,horz_length::Float64,vert_length::Float64,z_length::Float64,z_min::Float64,fs::Int=100)
  (sample_domain.Nx < 3) && error("Need more vertical lines. the sample domain 
    should have 2 more than number of planks in a given direction")
  (sample_domain.Ny < 3) && error("Need more horizontal lines. the sample domain 
    should have 2 more than number of planks in a given direction")

  num_horz_sample_points = round.(Int,ceil(fs*horz_length))
  num_vert_sample_points = round.(Int,ceil(fs*vert_length))

  vert_min = sample_domain.minmin.y
  vert_max = sample_domain.maxmax.y
  horz_min = sample_domain.minmin.x
  horz_max = sample_domain.maxmax.x
  
  unscaled_horz_curves, unscaled_vert_curves = sample_grid_func(f,sample_domain.Nx, sample_domain.Ny, num_vert_sample_points, num_horz_sample_points, vert_min, vert_max,horz_min, horz_max)

  horz_x_coords = map(p -> p.x, unscaled_horz_curves[1]) # Assuming the domain is the same accross horizontal curves
  vert_x_coords = map(p -> p.x, unscaled_vert_curves[1]) # Assuming the domain is the same accross horizontal curves
  
  # The following values will be used to compute the sample points and cut points
  rescale(vec,scl) = scl * ((vec - minimum(vec))/(maximum(vec) - minimum(vec)))
  scaled_horz_x_vals = rescale(horz_x_coords, horz_length)
  scaled_vert_x_vals = rescale(vert_x_coords, vert_length)

  vert_curves = Curves(length(unscaled_vert_curves))
  horz_curves = Curves(length(unscaled_horz_curves))

  for (i,c) in enumerate(unscaled_horz_curves)
    horz_func_vals = map(p -> p.y, c)
    crv = Vector{Point}()
    for (px,py) in zip(scaled_horz_x_vals, horz_func_vals)
      push!(crv, Point(px,py))
    end
    horz_curves[i] = crv
  end

  for (i,c) in enumerate(unscaled_vert_curves)
    vert_func_vals = map(p -> p.y, c)
    crv = Vector{Point}()
    for (px,py) in zip(scaled_vert_x_vals, vert_func_vals)
      push!(crv, Point(px,py))
    end
    vert_curves[i] = crv
  end
  
  glowify_curves_on_grid(vert_curves, horz_curves, mat_width, horz_length, vert_length, z_length, z_min)
end

#       5) translate the curves so that they don't overlap
#       6) export to svg

#function disjointify(planks)

#function export_to_svg(disjoint_planks,filename)



glowify_grid_func (generic function with 2 methods)

In [38]:
c = gensquare()
mapreduce(p -> p.x, min, c)

-0.5

In [34]:
c = Curves(5)

5-element Array{Array{PointsNCurves.Point,1},1}:
 #undef
 #undef
 #undef
 #undef
 #undef

In [24]:
c[2] = Curve(gensquare())

5-element Array{PointsNCurves.Point,1}:
 PointsNCurves.Point(-0.5, -0.5)
 PointsNCurves.Point(-0.5, 0.5) 
 PointsNCurves.Point(0.5, 0.5)  
 PointsNCurves.Point(0.5, -0.5) 
 PointsNCurves.Point(-0.5, -0.5)

LoadError: [91mUndefRefError: access to undefined reference[39m

In [22]:
typeof(gensquare()) == ClosedCurve && typeof(Curve(gensquare())) == Curve


true

In [ ]:
x_length = 7
y_length = 7
z_min = 2 # The function values will be shifted so that they have this minimum_value
z_length = 4 # The approximate difference between the max and min of the function. 

dc = .1 # approximate distance between neighboring sample points

